This is a jupyter notebook detailling how to attack the compression function in LPR-like primitives.

If you wish to perform a different attack using similar distinguishers, you can modify the 'create_tab' function.

Throughout this work, the results returned are the INDEXES of the dataframe. To compute the corresponding bias, using the following formula:

bias = + q*(index+1)/2^d

/ _ ! _ \ Warning / _ ! _ \ 

This version of the notebook does not have a fully recursive implementation of the adaptive attack. Thus, the adaptive attack only has a depth of 2 choices.

In [1]:
import pandas as pd #for dataframe
import itertools as iter #for combinatorix

#Hamming Weight
def hw(a):
    return bin(a).count('1')

#Creates the table of the difference between the HW of the line index 
# and the HW of the line index + the column index modulus the number of lines
def create_tab(d):
    siz = pow(2,d)
    return [[hw((j+i+1)%siz)-hw(i) for j in range(siz-1)]for i in range(siz)]

#Variant of create_tab which outputs a pandas data frame
def create_tab_pd(d):
    dat = create_tab(d)
    return pd.DataFrame(dat)

#Creates a list of all possible combinations of e elements of a list of size siz
def list_combi(siz,e):
    finlis=[]
    elem=list(range(0,siz-1))
    combi=iter.combinations(elem,e)
    for c in combi:
        lis=[]
        for sc in c:
            lis.append(sc)
        finlis.append(lis)    
    return finlis

#Returns a list of column index combinations for which there is no duplicated lines 
# when considering these specific combinations of columns 
def no_dup_list(df,e):
    finlis=[]
    com = list_combi(df.shape[1],e)
    for c in com:
        if df.iloc[:, lambda df: c].duplicated().any() == False:
            finlis.append(c)
    return finlis

#For a specific index i, returns a list of column indexes such as there is no duplicated 
# lines when considering these specific columns and the one of index i
def no_dup_no_comb(df,i):
    finlis=[]
    com= []
    for j in range(0,df.shape[1]):
        if j!=i:
            com.append([i,j])
    for c in com:
        if df.iloc[:, lambda df: c].duplicated().any() == False:
            finlis.append(c)
    return finlis

#Computes the number of bits recovered from a list of values: 
# pos computes the number of bits certain to be 1 with the HW of the AND of the values 
# neg computes the number of bits certain to be 0 with the HW of the AND of the NOT of the values
def bit_recovery(d,l):
    mask=pow(2,d)-1
    pos = mask
    neg = pos
    for i in l:
        pos = i & pos
        neg = neg & (i^mask)
    return hw(pos) + hw(neg)

#Returns the list of average bit recovery for every bias for a specific parameter d
def average_recovery(d):
    df = create_tab_pd(d)
    siz = pow(2,d)
    av_list = []
    for i in range(0,siz-1):
        av=0
        valu = df[i].drop_duplicates().values
        for v in valu:
            df_val=df[df[i]==v]
            valind=df_val.index.values.astype(int)
            res = bit_recovery(d,valind)
            av += res*df_val.shape[0]
        av_list.append([i,av/siz])
    return av_list

#Outputs the list of all possible attacks of e biases against the parameter d
def straight_attack(d,e):
    df = create_tab_pd(d)
    lis = no_dup_list(df,e)
    if not lis:
        print("There is no complete recovery for d = %d and only %d attempts"%(d,e))
    else:
        print("An example of complete recovery for d=%d in %d attempts is:"%(d,e))
        print(df.iloc[:,lambda df:lis[0]])

#Ouputs the first possible adaptive attack of depth 2 by increasing order of indexes 
# against the parameter d 
def adaptive_attack(d):
    df = create_tab_pd(d)
    for i in range(0,df.shape[1]):
        val_list=[]
        valu = df[i].drop_duplicates().values
        check = True
        for v in valu:
            df_val = df[df[i]==v]
            list = no_dup_no_comb(df_val,i)
            if not list:
                check &= False
                break
            else:
                val_list.append([v,list[0]])
        if check:
            print(i)
            for v in val_list:
                df_v = df[(df[i]==v[0])]
                print(df_v.iloc[:,lambda df_v: v[1]])
            return val_list
    return []

Chosen-ciphertext non-adaptative attack against Frodo-976. Returns a set of biases to use to perform a complete message recovery in only 2 guesses.

In [2]:
straight_attack(3,2)

An example of complete recovery for d=3 in 2 attempts is:
   1  4
0  1  2
1  1  1
2  0  2
3  0 -2
4  1  0
5  1 -1
6 -2  0
7 -2 -2


Chosen-ciphertext non-adaptative attack against Frodo-1344. Highlights there is no possible complete message recovery in only 2 guesses

In [3]:
straight_attack(4,2)

There is no complete recovery for d = 4 and only 2 attempts


Chosen-ciphertext non-adaptative attack against Frodo-1344. Returns an example of 3 biases to test for a complete message recovery

In [4]:
straight_attack(4,3)

An example of complete recovery for d=4 in 3 attempts is:
    0  3  9
0   1  1  2
1   0  1  2
2   1  1  1
3  -1  1  1
4   1  0  2
5   0  0  2
6   1  0 -2
7  -2  0 -2
8   1  1  0
9   0  1  0
10  1  1 -1
11 -1  1 -1
12  1 -2  0
13  0 -2  0
14  1 -2 -2
15 -4 -2 -2


Chosen-ciphertext adaptative attack against Frodo-1344. Returns an example of a first guess followed by a list of possible guesses depending on the first result

In [5]:
adaptive_attack(4)

4
    4  9
0   2  2
2   2  1
8   2  0
10  2 -1
   4  6
1  1  0
4  1  2
6  1  1
9  1 -2
    4  1
3  -1  0
7  -1 -1
12 -1  1
14 -1 -3
   4  0
5  0  0
    4  0
11 -3 -1
15 -3 -4
    4  0
13 -2  0


[[np.int64(2), [4, 9]],
 [np.int64(1), [4, 6]],
 [np.int64(-1), [4, 1]],
 [np.int64(0), [4, 0]],
 [np.int64(-3), [4, 0]],
 [np.int64(-2), [4, 0]]]

Evaluation of the average bit recovery for each bias in the Frodo-640 case

In [6]:
average_recovery(2)

[[0, 1.5], [1, 1.0], [2, 1.5]]

Evaluation of the average bit recovery for each bias in the Frodo-976 case

In [7]:
average_recovery(3)

[[0, 1.75], [1, 1.5], [2, 2.25], [3, 1.0], [4, 2.25], [5, 1.5], [6, 1.75]]

Evaluation of the average bit recovery for each bias in the Frodo-1344 case

In [8]:
average_recovery(4)

[[0, 1.875],
 [1, 1.75],
 [2, 1.875],
 [3, 1.5],
 [4, 1.375],
 [5, 2.25],
 [6, 1.875],
 [7, 1.0],
 [8, 1.875],
 [9, 2.25],
 [10, 1.375],
 [11, 1.5],
 [12, 1.875],
 [13, 1.75],
 [14, 1.875]]